In [2]:
import pandas as pd

# Preprocess
We leverage Google Research's synthetic word generator (published as part of the WinoDict project) to generate a large number of synthetic words. We then combine this with a high quality public dataset that contains words with their associated definition and a few usage examples. We use join the synthetic words with `word_def_usage` dataset to replace the real words with synthetic words to create a new dataset `word_def_usage_synthetic` that contains synthetic words with their associated definition and usage examples.

In [3]:
word_def_usage_path = "../data/word-meaning-examples.csv"
synthetic_words_path = "../data/words_300k.tsv"

In [4]:
# Load data
word_def_usage_df = pd.read_csv(word_def_usage_path)

word_def_usage_df = word_def_usage_df.dropna(
    subset=["Word", "Meaning", "Examples/0"]    # Drop rows with missing values (without at least one example), reset index
).reset_index(drop=True)
word_def_usage_df.shape

(13143, 12)

In [5]:
# 'Word's that are not unique
word_def_usage_df[word_def_usage_df['Word'].duplicated(keep=False)].shape

(866, 12)

In [6]:
synthetic_words_df = pd.read_csv(synthetic_words_path, sep='\t', names=['Word', 'Score', 'Rules'])
synthetic_words_df = synthetic_words_df.dropna().reset_index(drop=True)
synthetic_words_df.shape

(55084, 3)

In [14]:
# replace words in word_def_usage_df with words from synthetic_words_df
word_def_usage_synthetic_df = word_def_usage_df.copy()


def replace_word(i: int):
    # replace word
    word_def_usage_synthetic_df.Word[i] = synthetic_words_df.Word[i]

    # replace word in meaning
    word_def_usage_synthetic_df.Meaning[i] = str(
        word_def_usage_synthetic_df.Meaning[i]
    ).replace(
        str(word_def_usage_df.Word[i]),
        str(synthetic_words_df.Word[i]),
    )

    # replace word in examples
    for ei in range(10):
        example_col_id = f"Examples/{ei}"
        if word_def_usage_synthetic_df[example_col_id][i] != "nan":
            word_def_usage_synthetic_df[example_col_id][i] = str(
                word_def_usage_synthetic_df[example_col_id][i]
            ).lower().replace(
                str(word_def_usage_df.Word[i]).lower(),
                str(synthetic_words_df.Word[i]).lower(),
            )

word_def_usage_df.index.map(lambda i: replace_word(i))
word_def_usage_synthetic_df.head()

,Word,Meaning,Examples/0,Examples/1,Examples/2,Examples/3,Examples/4,Examples/5,Examples/6,Examples/7,Examples/8,Examples/9
0,undes,a phrase used to reference a situation that is...,the poor orphan’s life hasn’t been undes or ea...,"because mark studied for many weeks, taking th...",the stressed business owner quickly realized t...,nan,nan,nan,nan,nan,nan,nan
1,undel,something that is believed to be true without ...,religious people have the undel belief that go...,the jaded woman made undel assumptions that al...,christopher columbus had the undel belief that...,people make undel assumptions that the sun wil...,lawyers use undel arguments about their defend...,nan,nan,nan,nan,nan
2,undef,"a large, long-eared nocturnal animal that uses...",the long-eared undef slept all day and ate ins...,"with its rabbit like ears and piglike snout, t...","living a solitary nightlife, the african undef...",the undef is a snouty mammal that dines heavil...,"though it is a small mammal, the closest relat...",nan,nan,nan,nan,nan
3,overy,in amazement; shocked,i was taken overyby the server’s rude comment ...,"while amy enjoyed the film as a whole, as a mi...",ken believed he had a great marriage so he was...,"at seventy years of age, wanda was taken overy...",although i had been exercising daily and eatin...,nan,nan,nan,nan,nan
4,overb,toward the stern; behind,the wind was overb which made the ship glide f...,they placed the lifeboats in the rear part of ...,my first sailing lesson was learning that over...,"the sea spray was coming from overb, making my...",i was told to sit overb to lessen my seasickne...,nan,nan,nan,nan,nan


In [44]:
i = 3738
print(word_def_usage_synthetic_df["Examples/0"][i])
print(word_def_usage_synthetic_df["Examples/1"][i])
print(word_def_usage_synthetic_df["Examples/2"][i])
print(word_def_usage_synthetic_df["Examples/3"][i])
print(word_def_usage_synthetic_df["Examples/4"][i])
# print(word_def_usage_synthetic_df["Meaning"][i])

since the students want to impress their teacher, they phast perfect behavior in their class.
animals typically phast odd behaviors when they are put in uncomfortable situations.
it was heartbreaking to see the dog phast such fear by cowering in the corner when its abusive owner came into the room.
young children normally phast shyness around strangers they don’t know.
the angry fans phast rudeness and disgust toward the rival team by insulting and throwing litter at them.


In [45]:
print(word_def_usage_synthetic_df["Meaning"][i])

to purposefully show or reveal something


In [46]:
word_def_usage_df["Word"][i]

'Exhibit'

In [47]:
sum(word_def_usage_synthetic_df['Examples/4'] != 'nan')

11410

In [108]:
# split data into train, test and prompt sets
train = 0.5
test = 0.2
prompt = 0.3

train_df = word_def_usage_synthetic_df.sample(frac=train, random_state=42)
test_df = word_def_usage_synthetic_df.drop(train_df.index).sample(frac=test / (1 - train), random_state=42)
prompt_df = word_def_usage_synthetic_df.drop(train_df.index).drop(test_df.index).sample(frac=prompt / (1 - train - test), random_state=42)

train_df.shape, test_df.shape, prompt_df.shape

((6572, 12), (2628, 12), (3943, 12))

In [109]:
# verify that there are no duplicates in train, test and prompt sets
print(train_df[train_df.duplicated(subset=['Word', 'Meaning'], keep=False)].shape)
print(test_df[test_df.duplicated(subset=['Word', 'Meaning'], keep=False)].shape)
print(prompt_df[prompt_df.duplicated(subset=['Word', 'Meaning'], keep=False)].shape)

(0, 12)
(0, 12)
(0, 12)


In [110]:
# write to csv
train_df.to_csv("../out/word_def_usage_synthetic_train.csv", index=False)
test_df.to_csv("../out/word_def_usage_synthetic_test.csv", index=False)
prompt_df.to_csv("../out/word_def_usage_synthetic_prompt.csv", index=False)